<a href="https://colab.research.google.com/github/DavidAntonio7/DavidAntonio7-quero-data-team-training-2/blob/main/desafio5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder.getOrCreate()

In [39]:
df = spark.read.csv("sales_data.csv", header=True, inferSchema=True)


In [67]:
df.show(20)

+---------------+-----------------+--------------+-----------------+--------------------+-------------+-----------------------+-----------------+-------------+-------------+------------------------+------------+
|      user_name|user_country_name|user_city_name|     product_name| product_description|product_brand|product_value_in_dolars|       store_name|store_country|   store_city|datetime_of_the_purchase|buyed_amount|
+---------------+-----------------+--------------+-----------------+--------------------+-------------+-----------------------+-----------------+-------------+-------------+------------------------+------------+
|      Mark King|               UK|     Sheffield|            Dryer|Electric dryer wi...|      Samsung|                 699.99|         Waitrose|           UK|     Brighton|    2024-10-18 02:05:...|           4|
|   Alice Hughes|               UK|       Bristol|     Robot Vacuum|Smart robot vacuu...|       iRobot|                 299.99|  Supercheap Auto|    Aus

In [41]:
df_user = (
    df
    .select('user_name','user_country_name', 'user_city_name')
    .dropDuplicates()
    .withColumn('user_id', F.monotonically_increasing_id())
)

df_product = (
    df
    .select('product_name','product_description','product_brand','product_value_in_dolars')
    .dropDuplicates()
    .withColumn('product_id', F.monotonically_increasing_id())
)

df_store = (
    df
    .select('store_name','store_country','store_city')
    .dropDuplicates()
    .withColumn('store_id', F.monotonically_increasing_id())
)

In [42]:
df_with_ids = (
    df
    .join(df_user,    ['user_name','user_country_name', 'user_city_name'],                              'inner')
    .join(df_product, ['product_name','product_description','product_brand','product_value_in_dolars'], 'inner')
    .join(df_store,   ['store_name','store_country','store_city'],                                      'inner')
)

In [52]:
df_purchase = (
    df_with_ids
    .select('user_id','product_id','store_id','datetime_of_the_purchase','buyed_amount')
)

# Dimension tables

In [47]:
df_user.show(5)

+--------------+-----------------+--------------+-------+
|     user_name|user_country_name|user_city_name|user_id|
+--------------+-----------------+--------------+-------+
|  Sandra Green|           Canada|       Halifax|      0|
|Cheryl Russell|               UK|    Birmingham|      1|
|    Diana Reed|        Australia|        Mackay|      2|
|  Alice Hughes|        Australia|    Townsville|      3|
|  Megan Hughes|              USA|       Buffalo|      4|
+--------------+-----------------+--------------+-------+
only showing top 5 rows



In [48]:
df_product.show(5)

+-------------+--------------------+-------------+-----------------------+----------+
| product_name| product_description|product_brand|product_value_in_dolars|product_id|
+-------------+--------------------+-------------+-----------------------+----------+
|     E-reader|Lightweight e-rea...|       Kindle|                  89.99|         0|
|  Rice Cooker|Automatic rice co...|    Zojirushi|                 129.99|         1|
|     Keyboard|Mechanical keyboa...|        Razer|                 129.99|         2|
|Cutting Board|Bamboo cutting bo...|    John Boos|                  39.99|         3|
|   VR Headset|Virtual reality h...|       Oculus|                 299.99|         4|
+-------------+--------------------+-------------+-----------------------+----------+
only showing top 5 rows



In [49]:
df_store.show(5)

+-----------+-------------+-----------+--------+
| store_name|store_country| store_city|store_id|
+-----------+-------------+-----------+--------+
|Giant Tiger|       Canada|Quebec City|       0|
|     Lowe's|          USA| Fort Worth|       1|
|     Newegg|          USA|  San Diego|       2|
| John Lewis|           UK|    Glasgow|       3|
|      Boots|           UK|    Cardiff|       4|
+-----------+-------------+-----------+--------+
only showing top 5 rows



#Fact tables

In [54]:
df_purchase.show(5)

+-------+----------+--------+------------------------+------------+
|user_id|product_id|store_id|datetime_of_the_purchase|buyed_amount|
+-------+----------+--------+------------------------+------------+
|     18|        41|      60|    2024-10-18 02:05:...|           4|
|     25|        16|      52|    2024-12-10 17:22:...|           2|
|     32|        25|      63|    2025-01-09 16:15:...|           2|
|     33|        34|       1|    2024-10-25 03:02:...|           3|
|     83|        27|      46|    2024-11-02 11:20:...|           1|
+-------+----------+--------+------------------------+------------+
only showing top 5 rows



# questoes

- Quais foram os países com maior número de vendas ?

In [65]:
(
    df_purchase
    .join(df_store,'store_id', 'inner')
    .groupBy('store_country')
    .agg(F.count(F.col('store_id')).alias('numero_vendas'))
    .orderBy('numero_vendas', ascending = False)
).show()

+-------------+-------------+
|store_country|numero_vendas|
+-------------+-------------+
|       Canada|        13035|
|    Australia|        12469|
|          USA|        12271|
|           UK|        12225|
+-------------+-------------+



- Quais países tem mais lojas ?

In [66]:
(
    df_store
    .groupBy('store_country')
    .agg(F.count(F.col('store_id')).alias('numero_lojas'))
    .orderBy('numero_lojas', ascending = False)
).show()

+-------------+------------+
|store_country|numero_lojas|
+-------------+------------+
|       Canada|          18|
|          USA|          17|
|           UK|          17|
|    Australia|          17|
+-------------+------------+



- Quais países tiveram o maior valor total bruto de vendas ?


In [72]:
(
    df_purchase
    .join(df_store,'store_id', 'inner')
    .join(df_product,'product_id','inner')
    .withColumn('price', F.col('product_value_in_dolars') *  F.col('buyed_amount'))
    .groupBy('store_country')
    .agg(F.sum('price').alias('total_bruto'))
    .orderBy('total_bruto', ascending = False)
).show()

+-------------+-----------------+
|store_country|      total_bruto|
+-------------+-----------------+
|       Canada| 7246033.97000056|
|    Australia|6896470.250000497|
|          USA|6860512.980000538|
|           UK|6808784.100000538|
+-------------+-----------------+

